In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/[your_folder]/

In [ ]:
from __future__ import print_function
import os
import glob
import numpy as np
import skimage.io as io
import skimage.transform as trans


from keras.preprocessing.image import ImageDataGenerator
from keras import backend as keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [ ]:
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            # For one pixel in the image, find the class in mask and convert it into one-hot vector
            #   index = np.where(mask == i)
            #   index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #   new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)



def trainGenerator(batch_size,
                   train_path,
                   image_folder,
                   mask_folder,
                   aug_dict,
                   image_color_mode = "grayscale",
                   mask_color_mode = "grayscale",
                   image_save_prefix  = "image",
                   mask_save_prefix  = "mask",
                   flag_multi_class = False,
                   num_class = 2,
                   save_to_dir = None,
                   target_size = (256,256),
                   seed = 1):
    '''
    Genera maschere e immagini contemporanemante.
    Usare lo stesso seed per immagini (image_datagen) e
    maschere (mask_dategen) per eseguire lo stesso tipo trasformazioni.
    Per visualizzare i risultati, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)


# Legge dal path delle immagini, applica normalizzazione, resize e reshape
def testGenerator(test_path,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    for i in range(num_image):
        img = io.imread(os.path.join(test_path,"%d.png"%i),as_gray = as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img


def geneTrainNpy(image_path,mask_path,flag_multi_class = False,num_class = 2,image_prefix = "image",mask_prefix = "mask",image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(os.path.join(image_path,"%s*.png"%image_prefix))
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = io.imread(item,as_gray = image_as_gray)
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix),as_gray = mask_as_gray)
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr


def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255



def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

In [ ]:
def unet(pretrained_weights = None,
         input_size = (256,256,1)):
    inputs = Input(input_size)
    # 4 layer di convoluzioni
    # 2 layer 2x2
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    # layer di maxPooling
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottleneck layer
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    # Inizio dell' up-sampling (utilizzando la conv2D(UpSampling))
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    # Concateno sampling con il dropout
    merge6 = concatenate([drop4,up6], axis = 3) # skip connection mitiga la perdita di informazione nel down-sampling
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    # traduzione dei valori in probabilità, sigmoide per classificazione binaria
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(learning_rate = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    model.summary()

    if(pretrained_weights):
      model.load_weights(pretrained_weights)

    return model

## Train your Unet with membrane data with data augmentation
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

Link to dataset: https://drive.google.com/drive/folders/1_NZ8bro3qhDmg4cO0K1Of-w2Z4fiFZ30?usp=drive_link

In [ ]:
# In caso di daset di train molto piccolo applico trasformazioni
data_gen_args = dict(rotation_range=0.2, # probabilità con cui verrà applicata
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

myGenerator = trainGenerator(20,'data/membrane/train','image','label', data_gen_args, save_to_dir = "data/membrane/train/aug")

# Possibile visualizzare le 60 immagini e relative maschere su cui sono state applicate le trasformazioni in data/membrane/train/aug
num_batch = 3
for i,batch in enumerate(myGenerator):
    if(i >= num_batch):
        break

myGene = trainGenerator(2,'./data/membrane/train','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_aug_membrane.hdf5', monitor='loss', verbose=1, save_best_only = True)
model.fit(myGene, steps_per_epoch=2000, epochs=5, callbacks=[model_checkpoint])

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
2000/2000 [==============================] - ETA: 0s - loss: 0.2545 - accuracy: 0.9077
Epoch 1: loss improved from inf to 0.25446, saving model to unet_aug_membrane.hdf5
2000/2000 [==============================] - 388s 187ms/step - loss: 0.2545 - accuracy: 0.9077
Epoch 2/5
2000/2000 [==============================] - ETA: 0s - loss: 0.1963 - accuracy: 0.9389
Epoch 2: loss improved from 0.25446 to 0.19632, saving model to unet_aug_membrane.hdf5
2000/2000 [==============================] - 377s 188ms/step - loss: 0.1963 - accuracy: 0.9389
Epoch 3/5
2000/2000 [==============================] - ETA: 0s - loss: 0.1677 - accuracy: 0.9515
Epoch 3: loss improved from 0.19632 to 0.16774, saving model to unet_aug_membrane.hdf5
2000/2000 [==============================] - 375s 188ms/step - loss: 0.1677 - accuracy: 0.9515
Epoch 4

In [ ]:
testGene = testGenerator("data/membrane/test")
model = unet()
model.load_weights("unet_aug_membrane.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("data/membrane/test_aug",results)

## Train your Unet with membrane data without data augmentation
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

In [ ]:
data_gen_args = dict()

In [ ]:
myGene = trainGenerator(2,'./data/membrane/train','image','label',data_gen_args,save_to_dir = None)
model = unet()
# Monitora andamento del training rispetto ad 1+ parametri scelti, salva il modello rispetto ai parametri
model_checkpoint = ModelCheckpoint('unet_membrane_no_aug.hdf5', monitor='loss', verbose=1, save_best_only = True)
# Train modello, a differenza di PyTorch devo specificare il numero di step per epoche
model.fit(myGene, steps_per_epoch=2000, epochs=5, callbacks=[model_checkpoint])

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
2000/2000 [==============================] - ETA: 0s - loss: 0.1209 - accuracy: 0.9501
Epoch 1: loss improved from inf to 0.12092, saving model to unet_membrane_no_aug.hdf5
2000/2000 [==============================] - 381s 189ms/step - loss: 0.1209 - accuracy: 0.9501
Epoch 2/5
2000/2000 [==============================] - ETA: 0s - loss: 0.0132 - accuracy: 0.9951
Epoch 2: loss improved from 0.12092 to 0.01324, saving model to unet_membrane_no_aug.hdf5
2000/2000 [==============================] - 375s 188ms/step - loss: 0.0132 - accuracy: 0.9951
Epoch 3/5
2000/2000 [==============================] - ETA: 0s - loss: 0.0059 - accuracy: 0.9980
Epoch 3: loss improved from 0.01324 to 0.00588, saving model to unet_membrane_no_aug.hdf5
2000/2000 [==============================] - 375s 187ms/step - loss: 0.0059 - accuracy: 0.9980
Epoch 4/5
2000/2000 [==============================] - ETA: 0s - loss: 0.0026 

In [ ]:
testGene = testGenerator("./data/membrane/test")
model = unet()
model.load_weights("unet_membrane_no_aug.hdf5") # caricare i pesi salvati durante il training
results = model.predict_generator(testGene,30,verbose=1)
saveResult("./data/membrane/test_no_aug",results)